# Process the data obtained from polarimetry and reorganize the data into the correct folders

In [1]:
%load_ext autoreload
%autoreload 2

import os
from processingmm import batch_processing, libmpMuelMat

In [6]:
# set the parameters to run the script
directories = [r'C:\Users\romai\Documents\program_files\reconstructor\pat10-sec1']
calib_directory = r'C:\Users\romai\Documents\program_files\reconstructor\calibration'

In [7]:
# set run_all to true in order to run the pipeline on all the folders (even the ones already processed)
# set the parameters to be used for the line visualisation with the parameter parameter_set
batch_processing.batch_process(directories, calib_directory, run_all = False, parameter_set = 'GCB_symposium_Fixed')

100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [13:19<00:00, 34.75s/it]


['C:\\Users\\romai\\Documents\\program_files\\reconstructor\\pat10-sec1\\dev\\2023-08-31_T_PAT10-SEC1-C3_FX_M_1',
 'C:\\Users\\romai\\Documents\\program_files\\reconstructor\\pat10-sec1\\dev\\2023-08-31_T_PAT10-SEC1-F2_FX_M_2',
 'C:\\Users\\romai\\Documents\\program_files\\reconstructor\\pat10-sec1\\dev\\2023-08-31_T_PAT10-SEC1-B1_FX_M_1',
 'C:\\Users\\romai\\Documents\\program_files\\reconstructor\\pat10-sec1\\dev\\2023-09-01_T_PAT10-SEC1-F1_FX_M_1',
 'C:\\Users\\romai\\Documents\\program_files\\reconstructor\\pat10-sec1\\dev\\2023-08-31_T_PAT10-SEC1-D1_FX_M_1',
 'C:\\Users\\romai\\Documents\\program_files\\reconstructor\\pat10-sec1\\dev\\2023-08-31_T_PAT10-SEC1-B2_FX_M_1',
 'C:\\Users\\romai\\Documents\\program_files\\reconstructor\\pat10-sec1\\dev\\2023-09-01_T_PAT10-SEC1-B5_FX_M_1',
 'C:\\Users\\romai\\Documents\\program_files\\reconstructor\\pat10-sec1\\dev\\2023-09-01_T_PAT10-SEC1-C6_FX_M_1',
 'C:\\Users\\romai\\Documents\\program_files\\reconstructor\\pat10-sec1\\dev\\2023-09-01